# IS 4487 Lab 3

## Outline

Repeat exercises from Lab 2, but with the *SuperStore Retail Orders* dataset
- Loading installed packages
- Inspect data
- Explore plot types

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/Scripts/lab_03_retailer_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Superstore Data Dictionary

 ORDER VARIABLES:
 - Order ID
 - Order Date
 - Order Year Month - Year and month of the order
 - Order Type - Was the order completed at a store or online? (Retail, Online)
 - Quantity - Quantity ordered for the product

 CUSTOMER VARIABLES:
 - Customer Name
 - City
 - State Province
 - Email

PRODUCT VARIABLES:
 - Product Name
 - Product Line - Category of the product (i.e. Bikes Phones)
 - Product Price - Price in US Dollars
 - Product Status - Current status of the product (Active, Inactive)

## Load Libraries

➡️ Assignment Tasks
- Load any necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import Data into Dataframe

➡️ Assignment Tasks
- Import data from the SuperStore retail dataset into a dataframe (in GitHub go to Labs > DataSets)
- Describe or profile the dataframe

In [9]:
url = 'https://github.com/Stan-Pugsley/is_4487_base/blob/main/Labs/DataSets/superstore_retail_orders.csv?raw=true'
df = pd.read_csv(url, on_bad_lines='skip')

print (df.head())
print (df.info())
print (df.describe())

   order_id  order_date order_year_month  customer_name               city  \
0     51188  2023-07-01          2023-07  Audrey Blanco  North Cherylmouth   
1     51187  2023-07-01          2023-07    Isaac Allen         Traceyland   
2     51199  2023-07-02          2023-07     Ian Morgan          Dylanberg   
3     51241  2023-07-04          2023-07     Alexa Bell     Lake Tracytown   
4     51297  2023-07-08          2023-07     Casey Diaz        Byrdchester   

  state_province                    email order_type  quantity  \
0  New Hampshire  audrey_blanco@email.com     Retail       3.0   
1  West Virginia    isaac_allen@email.com     Online       4.0   
2   South Dakota     ian_morgan@email.com     Online       5.0   
3      Wisconsin     alexa_bell@email.com     Online       1.0   
4    Mississippi     casey_diaz@email.com     Online       1.0   

                                        product_name product_line  \
0  Apple iPhone XR (Yellow, 256 GB) (Includes Ear...       Mobile

## Prepare Data

➡️ Assignment Tasks
- Create a "revenue" variable and make sure it is a numeric datatype
- Remove rows with empty (NULL) values
- Locate outliers in at least two of the variables.   Filter them out if they are mistakes; keep them if they are exceptional cases.

In [11]:
df['revenue'] = df['quantity'] * df['product_price']
df['revenue'] = pd.to_numeric(df['revenue'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56043 entries, 0 to 56042
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          56043 non-null  int64  
 1   order_date        56011 non-null  object 
 2   order_year_month  56011 non-null  object 
 3   customer_name     56043 non-null  object 
 4   city              55569 non-null  object 
 5   state_province    55569 non-null  object 
 6   email             56043 non-null  object 
 7   order_type        56043 non-null  object 
 8   quantity          56004 non-null  float64
 9   product_name      56043 non-null  object 
 10  product_line      56043 non-null  object 
 11  product_price     56043 non-null  float64
 12  product_status    56043 non-null  object 
 13  revenue           56004 non-null  float64
dtypes: float64(3), int64(1), object(10)
memory usage: 6.0+ MB


In [14]:
df.dropna()
df.info()

print("Number of NULL values after dropping rows:")
print(df.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
Index: 55498 entries, 0 to 56042
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          55498 non-null  int64  
 1   order_date        55498 non-null  object 
 2   order_year_month  55498 non-null  object 
 3   customer_name     55498 non-null  object 
 4   city              55498 non-null  object 
 5   state_province    55498 non-null  object 
 6   email             55498 non-null  object 
 7   order_type        55498 non-null  object 
 8   quantity          55498 non-null  float64
 9   product_name      55498 non-null  object 
 10  product_line      55498 non-null  object 
 11  product_price     55498 non-null  float64
 12  product_status    55498 non-null  object 
 13  revenue           55498 non-null  float64
dtypes: float64(3), int64(1), object(10)
memory usage: 6.4+ MB
Number of NULL values after dropping rows:
order_id            0
order_date      

## Summarize Data

➡️ Assignment Tasks
- Display the average price per product line
- Display the total revenue for all months

In [ ]:
#average price

In [ ]:
#total revenue